# Stable Diffusion v 1.4
Implement Stable Diffusion via Huggingface diffusers

In [12]:
# Install dependencies
%pip install -Uq -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [21]:
with open('huggingface-access-tokens.txt', 'r') as f:
    YOUR_TOKEN = f.readlines()[0]

To retrive the access tokens, refer to [this documentation](https://huggingface.co/docs/hub/security-tokens).

[Link to get token](https://huggingface.co/settings/tokens).

In [14]:
import diffusers
print("diffusers version: ", diffusers.__version__)

diffusers version:  0.2.4


In [15]:
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
%%capture
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               revision = "fp16",
                                               torch_dtype = torch.float16,
                                               use_auth_token = YOUR_TOKEN)

In [32]:
pipe.to(torch_device)

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.2.4",
  "feature_extractor": [
    "transformers",
    "CLIPFeatureExtractor"
  ],
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
prompt = "a photograph of an astronaut riding a horse"

image = pipe(prompt)["sample"][0]

Deterministic output instead of random image generation.

In [ ]:
generator = torch.Generator("cuda").manual_seed(42)
image = pipe(prompt,
            guidance_scale = 7.5,
            generator = generator
            )["sample"][0]